In [1]:
import numpy as np
from scipy.linalg import sqrtm 
from IPython.display import HTML
from gcn_code.GCNLayer import GCNLayer
from gcn_code.GCNNetwork import GCNNetwork
import pickle as pk
import time
#%matplotlib inline

In [2]:
start_time = time.time()

In [3]:
city = 'Perth'

In [4]:
file = open('data_process/' + city + '/local', 'rb')
local_adj = pk.load(file)
file = open('data_process/' + city + '/global', 'rb')
global_adj = pk.load(file)

In [5]:
global_adj.shape

(25, 25)

In [6]:
def create_A_HAT(A):
    A_mod = A + np.eye(A.shape[0]) # add self-connections

    D_mod = np.zeros_like(A_mod)
    np.fill_diagonal(D_mod, np.asarray(A_mod.sum(axis=1)).flatten())


    D_mod_invroot = np.linalg.inv(sqrtm(D_mod))

    A_hat = D_mod_invroot @ A_mod @ D_mod_invroot
    
    return A_hat

In [7]:
global_adj = create_A_HAT(global_adj)

In [8]:
for item in local_adj:
    local_adj[item] = create_A_HAT(local_adj[item])

In [9]:
gcn_model = GCNNetwork(
    n_inputs=global_adj.shape[0], 
    n_outputs=global_adj.shape[0], 
    n_layers=3,
    hidden_sizes=[global_adj.shape[0], global_adj.shape[0], global_adj.shape[0]], 
    activation=np.tanh,
    seed=100,
)
global_adj = gcn_model.embedding(global_adj, np.eye(global_adj.shape[0]))

In [10]:
for item in local_adj:
    gcn_model = GCNNetwork(
        n_inputs=local_adj[item].shape[0], 
        n_outputs=local_adj[item].shape[0], 
        n_layers=3,
        hidden_sizes=[global_adj.shape[0], global_adj.shape[0], global_adj.shape[0]], 
        activation=np.tanh,
        seed=100,
    )
    local_adj[item] = gcn_model.embedding(local_adj[item], np.eye(local_adj[item].shape[0]))

In [11]:
np.around(global_adj[0], decimals=10, out=None)

array([-0.06364016, -0.06219697,  0.06766935, -0.02698081, -0.00958266,
       -0.11081271, -0.12703887,  0.02346425, -0.00894898, -0.06813898,
        0.06795832, -0.18444018, -0.11382544,  0.21903383,  0.00750152,
        0.0147644 ,  0.10598856, -0.1413845 , -0.1235894 ,  0.07163965,
       -0.03431186,  0.05316919, -0.07616762,  0.07389047,  0.06832545])

In [12]:
for item in local_adj:
    local_adj[item] = local_adj[item] + global_adj

In [13]:
with open("data_process/" + city + "/adjacency","wb") as file:
    pk.dump(local_adj, file)

In [14]:
final_time = time.time()

In [15]:
final_time - start_time

0.21325087547302246

In [16]:
f = open("results/" + city + "/time", "a")
f.writelines("convolution=" + str(final_time - start_time) + "\n")
f.close()